In [ ]:
using GridapGmsh
using Gmsh: gmsh
using Gridap
using Gridap.Geometry
using Gridap.TensorValues
using LineSearches: BackTracking
using LinearAlgebra

In [ ]:
const E_mat = 2e4
const ν_mat = 0.18

const G₁₂_mat = E_mat/(2*(1+ν_mat))

const λ_mat = E_mat*ν_mat/((1+ν_mat)*(1-2*ν_mat))
const μ_mat = E_mat/(2*(1+ν_mat))
const k_mat = λ_mat + μ_mat

In [ ]:
const β = 1.25
const σcI = 2.5
const σcII = β * σcI

const α = 1.25
const GcI = 0.09
const GcII = α * GcI

In [ ]:
const η = 1e-15

In [ ]:
const ls = 10

In [ ]:
mI(GcIc) = 4*GcIc*k_mat/(pi*ls*σcI^2)
mII(GcIIc) = 4*GcIIc*μ_mat/(pi*ls*σcII^2)

const ψ_Crit_I = σcI^2/(2*k_mat)
const ψ_Crit_II = σcII^2/(2*μ_mat)

In [ ]:
const  L = 500
const  H = 500
const  LOf = 220
const  hfc = ls/4 #Mesh  size  parameter
const  hfl = ls #Mesh  size  parameter
const  hf = ls/4 #Mesh  size  parameter
const  h = 10*hf #Mesh  size  parameter
const  thick = 100
const FMR = 1*ls
gmsh.initialize()
gmsh.option.setNumber("General.Terminal", 1)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, h ,1)
gmsh.model.geo.addPoint(L/2, 0.0, 0.0, h, 2)
gmsh.model.geo.addPoint(L/2, H/2, 0.0, hfc, 3)
gmsh.model.geo.addPoint(L/2+LOf-1.25*10, H/2, 0.0, hfl, 4)
gmsh.model.geo.addPoint(L/2+LOf+1.25*10, H/2, 0.0, hfl, 5)
gmsh.model.geo.addPoint(L, H/2, 0.0, h, 6)
gmsh.model.geo.addPoint(L, H, 0.0, h, 7)
gmsh.model.geo.addPoint(0.0, H, 0.0, h, 8)
gmsh.model.geo.addPoint(0.0, H/2, 0.0, hf, 9)

#gmsh.model.geo.addPoint(0.0, H/2, 0.0, hf, 9)

gmsh.model.geo.addLine(1, 2, 1)
gmsh.model.geo.addLine(2, 3, 2)
gmsh.model.geo.addLine(3, 4, 3)
gmsh.model.geo.addLine(4, 5, 4)
gmsh.model.geo.addLine(5, 6, 5)
gmsh.model.geo.addLine(6, 7, 6)
gmsh.model.geo.addLine(7, 8, 7)
gmsh.model.geo.addLine(8, 9, 8)
gmsh.model.geo.addLine(9, 1, 9)


gmsh.model.geo.addLine(3, 9, 100)

gmsh.model.geo.addCurveLoop([1,2,3,4,5,6,7,8,9],1)
gmsh.model.geo.addPlaneSurface([1], 1)

gmsh.model.addPhysicalGroup(2, [1],1)
gmsh.model.addPhysicalGroup(1, [1],1)
gmsh.model.addPhysicalGroup(1, [4],2)

gmsh.model.setPhysicalName(2, 1, "Domain")
gmsh.model.setPhysicalName(1, 1, "Support")
gmsh.model.setPhysicalName(1, 2, "LoadLine")

gmsh.model.mesh.field.add("Box", 11)
gmsh.model.mesh.field.setNumber(11, "VIn", hf)
gmsh.model.mesh.field.setNumber(11, "VOut", h)
gmsh.model.mesh.field.setNumber(11, "XMin", 0)
gmsh.model.mesh.field.setNumber(11, "XMax", L/2)
gmsh.model.mesh.field.setNumber(11, "YMin", 0.5*H-FMR)
gmsh.model.mesh.field.setNumber(11, "YMax", 0.6*H)
gmsh.model.mesh.field.setAsBackgroundMesh(11)

#=gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "EdgesList", [100])

gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "IField", 1)
gmsh.model.mesh.field.setNumber(2, "LcMin", hf)
gmsh.model.mesh.field.setNumber(2, "LcMax", h)
gmsh.model.mesh.field.setNumber(2, "DistMin", 25*ls)
gmsh.model.mesh.field.setNumber(2, "DistMax", 35*ls)
gmsh.model.mesh.field.setAsBackgroundMesh(2)=#

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(2)
gmsh.write("LShapedPanel.msh")
gmsh.finalize()

In [ ]:
model = GmshDiscreteModel("LShapedPanel.msh")
writevtk(model,"LShapedPanel")  

In [ ]:
I2 = SymTensorValue{2,Float64}(1.0 ,0.0 ,1.0)
I4 = I2⊗I2
I4_sym = one(SymFourthOrderTensorValue{2,Float64})
I4_vol = (1.0/2)*I4
I4_dev = I4_sym  - I4_vol  

In [ ]:
p = -0.5

In [ ]:
function degDer1(ϕ,GcI)
    mIc = mI(GcI)
    g = ((ϕ-1)*(ϕ*(2*p+1)+1)*mIc)/((ϕ*ϕ*(mIc*p+1)+(ϕ*(mIc-2)+1))^2) 
    return g
end

In [ ]:
function degDer2(ϕ,GcII)
    mIIc = mII(GcII)
    g = ((ϕ-1)*(ϕ*(2*p+1)+1)*mIIc)/((ϕ*ϕ*(mIIc*p+1)+(ϕ*(mIIc-2)+1))^2) 
    return g
end

In [ ]:
function σfun(ε,ε_in,s_in,GcI,GcII)
    mIc = mI(GcI)
    mIIc = mII(GcII)
    εdev = I4_dev ⊙ ε
    σI = k_mat*tr(ε)*one(ε)
    σII = 2*μ_mat*εdev
    σ = ((1-s_in)^2 / ((1-s_in)^2 + ((mIc*s_in*(1 + p*s_in)))))*(σI) + ((1-s_in)^2 / ((1-s_in)^2 + ((mIIc*s_in*(1 + p*s_in)))))*(σII)
    return  σ
end  

In [ ]:
function Eigen(ε)
    εArray = get_array(ε)
    Λ, P = eigen(εArray)
    ε1 = Λ[1]
    ε2 = Λ[2]
    if ε1 >= 0 &&  ε2 >= 0
        Λpos = [ε1 0; 0 ε2]
        Λneg = [0 0; 0 0]
    elseif ε1 >= 0 &&  ε2 < 0
        Λpos = [ε1 0; 0 0]
        Λneg = [0 0; 0 ε2]
    elseif ε1 < 0 &&  ε2 >= 0
        Λpos = [0 0; 0 ε2]
        Λneg = [ε1 0; 0 0]
    elseif ε1 < 0 &&  ε2 < 0
        Λpos = [0 0; 0 0]
        Λneg = [ε1 0; 0 ε2]
    end
    εPos = TensorValue(P*Λpos*P')
    εNeg = TensorValue(P*Λneg*P')
    return εPos, εNeg
end

In [ ]:
function ψPosTen(ε_in)
    εdev = I4_dev ⊙ ε_in
    εPos, εNeg = Eigen(εdev) 
    if tr(ε_in) > 0.0
        ψPlus = 0.5*k_mat*(tr(ε_in))^2
    else
        ψPlus = 0.0
    end
    return ψPlus
end

In [ ]:
function ψPosShear(ε_in)
    εdev = I4_dev ⊙ ε_in
    εPos, εNeg = Eigen(εdev) 
    ψPlus = μ_mat * (εPos ⊙ εPos)
    return ψPlus
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_in = ψhPos_in
  if ψPlus_in >= ψPlusPrev_in
    ψPlus_out = ψPlus_in
  else
    ψPlus_out = ψPlusPrev_in
  end
  true,ψPlus_out
end

In [ ]:
function EnergyState(ψPlusPrev_in,ψhPos_in)
  ψPlus_out = ψhPos_in
  true,ψPlus_out
end

In [ ]:
function  project(q,model ,dΩ,order)
    reffe = ReferenceFE(lagrangian ,Float64 ,order)
    V = FESpace(model ,reffe ,conformity =:L2)
    a(u,v) =∫(u*v)*dΩ
    b(v) =∫(v*q)*dΩ
    op = AffineFEOperator(a,b,V,V)
    qh = Gridap.solve(op)
    return  qh
end
     

In [ ]:
order = 1
reffe_PF = ReferenceFE(lagrangian ,Float64,order)
V0_PF = TestFESpace(model ,reffe_PF;conformity =:H1)
U_PF = TrialFESpace(V0_PF)
sh = zero(V0_PF)

In [ ]:
reffe_Disp = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
        V0_Disp = TestFESpace(model,reffe_Disp;
          conformity=:H1,
          dirichlet_tags=["Support","LoadLine"],
          dirichlet_masks=[(true,true), (false,true)])

uh = zero(V0_Disp)

In [ ]:
degree = 2*order
Ω= Triangulation(model)
dΩ= Measure(Ω,degree) 

In [ ]:
labels = get_face_labeling(model)
LoadTagId = get_tag_from_name(labels,"LoadLine")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
nls = NLSolver(
  show_trace=true,
  method=:newton,
  linesearch=BackTracking(), iterations = 10)
solver = FESolver(nls)

In [ ]:
function run_PF(x0,vApp,cache,GcI,GcII,ψTenPrev_in,ψShearPrev_in)
     U_PF = TrialFESpace(V0_PF)
     res(s,ϕ) = ∫((2/pi)*ls*∇(ϕ) ⋅ ∇(s) + (degDer1∘(s,GcI))*(ψTenPrev_in/GcI)*ϕ + (degDer2∘(s,GcII))*(ψShearPrev_in/GcII)*ϕ + ((1/pi)*(2-2*s)/ls)*ϕ)*dΩ
     op = FEOperator(res,U_PF,V0_PF)
     sh_out = FEFunction(U_PF,x0)
     sh_out, cache = solve!(sh_out,solver,op,cache)
  return sh_out, get_free_dof_values(sh_out), cache
end

In [ ]:
function  stepDisp(uh_in,sh_in,vApp,GcI,GcII)
    uApp1(x) = VectorValue(0.0,0.0)
    uApp2(x) = VectorValue(0.0,vApp)
    U_Disp = TrialFESpace(V0_Disp,[uApp1 ,uApp2])
    a_Disp(u,v) =∫( (ε(v)⊙(σfun∘(ε(u),ε(uh_in),sh_in,GcI,GcII)) ) )*dΩ
    b_Disp(v) = 0.0
    op_Disp = AffineFEOperator(a_Disp ,b_Disp ,U_Disp ,V0_Disp)
    uh_out = solve(op_Disp)
    return  uh_out
end

In [ ]:
function χFun1(ψhTenPos_in,ψhShearPos_in)
    χVal = (ψhTenPos_in/(ψhShearPos_in + ψhTenPos_in))^2
    return χVal
end

function χFun2(ψhTenPos_in,ψhShearPos_in)
    χVal = (ψhShearPos_in/(ψhShearPos_in + ψhTenPos_in))^2
    return χVal
end

In [ ]:
function GcFunI(χVal1)
    Gc = GcII + (GcI - GcII)*χVal1
    return Gc
end

function GcFunII(χVal2)
    Gc = GcI + (GcII - GcI)*χVal2
    return Gc
end

In [ ]:
function ψCritFun(χVal1,χVal2)
    ψCrit_I_in = χVal1 * ψ_Crit_I
    ψCrit_II_in = χVal2 * ψ_Crit_II
    return ψCrit_I_in,ψCrit_II_in
end

In [ ]:
vApp = 0.0
const vAppMax = 0.8
delv = vAppMax/100 #1e-4
innerMax = 10
count = 0
Load = Float64[]
Displacement = Float64[]
push!(Load, 0.0)
push!(Displacement, 0.0)

χValPrev1 = 0.5 
χValPrev2 = 0.5 

χVal1 = CellState(χValPrev1,dΩ)
χVal2 = CellState(χValPrev2,dΩ)

GcPrevI = GcII + (GcI - GcII)*χValPrev1
GcPrevII = GcI + (GcII - GcI)*χValPrev2

ψCrit_I_Prev = (χValPrev1)*ψ_Crit_I
ψCrit_II_Prev = (χValPrev2)*ψ_Crit_II

GcIV = project(CellState(GcPrevI,dΩ) ,model ,dΩ,order) 
GcIIV = project(CellState(GcPrevII,dΩ) ,model ,dΩ,order) 

ψTenPrev = CellState(ψCrit_I_Prev,dΩ)
ψShearPrev = CellState(ψCrit_II_Prev,dΩ)

x0_PF = zeros(Float64,num_free_dofs(V0_PF))
cache_1 = nothing

uh = zero(V0_Disp)
sh = zero(V0_PF)

sh_Prev = sh
uh_Prev = uh

while  vApp .< vAppMax
    count = count  .+ 1
    
    #=if vApp <= 0.25
        delv = 0.25/50
    else
        delv = vAppMax/20
end=#
    
    vApp = vApp .+ delv
    
    print("\n Entering  displacemtent  step$count :", float(vApp))
        
    for  inner = 1: innerMax
        
        ψhTenPrev = project(ψTenPrev ,model ,dΩ,order)
        ψhShearPrev = project(ψShearPrev ,model ,dΩ,order)
        
        RelErr = abs(sum(∫( (2/pi)*ls*∇(sh)⋅ ∇(sh) + (degDer1∘(sh,GcIV))*(ψhTenPrev/GcIV)*sh + (degDer2∘(sh,GcIIV))*(ψhShearPrev/GcIIV)*sh - (2/(pi*ls))*sh*sh)*dΩ + ∫( (2/(pi*ls))*sh)*dΩ))/abs(sum(∫( (2/(pi*ls))*sh)*dΩ))
 
        #RelErr = max(RelErr1,RelErr2)
        
        println("\nRelative error =", float(RelErr), "\n")
    
        sh,x0_PF,cache_1 = run_PF(x0_PF,vApp,cache_1,GcIV,GcIIV,ψhTenPrev,ψhShearPrev)
        uh = stepDisp(uh,sh,vApp,GcIV,GcIIV)
        
        ψhTenPos_in =ψPosTen∘(ε(uh))
        ψhShearPos_in =ψPosShear∘(ε(uh))
        
        update_state!(new_EnergyState ,ψTenPrev ,ψhTenPos_in)
        update_state!(new_EnergyState ,ψShearPrev ,ψhShearPos_in)
        
        χVal1 = χFun1∘(ψTenPrev,ψShearPrev)
        χVal2 = χFun2∘(ψTenPrev,ψShearPrev)
        
        GcIV = GcFunI(χVal1)
        GcIIV = GcFunII(χVal2)
        ψCrit_I_in,ψCrit_II_in = ψCritFun(χVal1,χVal2)
        
        
        update_state!(new_EnergyState ,ψTenPrev, ψCrit_I_in)
        update_state!(new_EnergyState ,ψShearPrev, ψCrit_II_in)
        
        if   RelErr  < 1e-4 || isnan(RelErr)
            break
        end
    end
    
    GcVal1 = project(GcIV ,model ,dΩ,order)
    GcVal2 = project(GcIIV ,model ,dΩ,order)
    
    χValC1 = project(χVal1 ,model ,dΩ,order)
    χValC2 = project(χVal2 ,model ,dΩ,order)
    Node_Force = sum(∫(n_Γ_Load⋅(σfun∘(ε(uh),ε(uh),sh,GcVal1,GcVal2)))*dΓ_Load)
    
    push!(Load , Node_Force[2])
    push!( Displacement , vApp)
    if mod(count,2) == 0
         writevtk(Ω,"results_SymThreePtBendingTest$count",cellfields=
        ["uh"=>uh,"s"=>sh , "epsi"=>ε(uh), "modefactorI"=>χValC1, "modefactorII"=>χValC2 ])
    end    
end

In [ ]:
using Plots

In [ ]:
plot(Displacement,Load*100/1e3)
#xlims!(0,0.006)

In [ ]:
using DelimitedFiles
using CSV
using DataFrames

In [ ]:
using LaTeXStrings

In [ ]:
open("load_dispPFCZMNewC$ls.csv", "w") do io
             writedlm(io, [Displacement Load])
end